In [29]:
import os
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

What should the assisstant do? 
1. We obv dont want it to just call gpt cause you can do that yourself 
2. We want it to effectively summarise contents of a page you provide 
3. Find appropriate links on that page and disgard irrelevant ones 
4. Go to those links and get the relevent information which contribiutes to your understanding 
5. Add that to the summary 
6. Organise the summary and show which information is coming from where. 

pip install requests python-dotenv beautifulsoup4 ipython openai

pip3 install requests python-dotenv beautifulsoup4 ipython openai

In [20]:
#the code will ask the user for a url
url = input("Enter the URL of the webpage you want to summarize: ")

In [30]:
# Load API Key
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

MODEL = "gpt-4o-mini"
openai = OpenAI(api_key=api_key)

#for ollama instead of openai
# import ollama
# ollama.chat(model="llama3.2", messages=[{"role": "user", "content": "Hello, how are you?"}])


In [22]:
# Headers for requests
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        self.links = [link.get('href') for link in soup.find_all('a') if link.get('href')]

In [23]:
# Summarize main page
system_prompt_main = "You are an assistant that analyzes a webpage and provides an executive summary on the main topic discussed. Your goal is to educate the reader about the subject matter, NOT the organization. Ignore company descriptions, advertisements, marketing. Respond in markdown."

def summarise_main_page(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_main},
            {"role": "user", "content": f"Website Title: {website.title}\n\nContent:\n{website.text[:5000]}"}
        ],
    )
    return response.choices[0].message

main_summary = summarise_main_page(url)

In [24]:
# Identify relevant links
system_prompt_links = "You are provided with a list of links from a webpage. Identify only those links that are relevant for understanding the main topic of the page. Ignore company-related pages like About, Services, or Careers. Respond in JSON format with full URLs."

def get_all_links(url):
    # Convert relative URLs to absolute URLs
    website = Website(url)
    base_url = url.rstrip('/')
    full_links = []
    
    for link in website.links:
        if link:
            if link.startswith('http'):
                full_links.append(link)
            elif link.startswith('//'):
                full_links.append('https:' + link)
            elif link.startswith('/'):
                full_links.append(base_url + link)
            elif not link.startswith('#'):
                full_links.append(base_url + '/' + link)
    
    print("All gathered links:", full_links)
    return full_links

links = get_all_links(url)  # First get all links

All gathered links: ['https://www.wsiworld.com', 'https://www.wsiworld.com/consultants', 'https://www.wsiworld.com/blog/how-ai-is-revolutionizing-seo/tel:905-678-7588', 'https://www.wsiworld.com', 'https://www.wsiworld.com/about-us/culture#video', 'https://www.wsiworld.com/about-us/why-wsi', 'https://www.wsiworld.com/about-us', 'https://www.wsiworld.com/about-us/leadership', 'https://www.wsiworld.com/about-us/awards', 'https://www.wsiworld.com/about-us/culture', 'https://www.wsiworld.com/our-services/leads-and-sales', 'https://www.wsiworld.com/our-services/website', 'https://www.wsiworld.com/our-services/ai-consulting', 'https://www.wsiworld.com/our-services/brand-awareness', 'https://www.wsiworld.com/our-services', 'https://www.wsiworld.com/speak-with-a-wsi-expert', 'https://www.wsiworld.com/customer-stories/case-studies', 'https://www.wsiworld.com/client-portfolio', 'https://www.wsiworld.com/customer-stories', 'https://www.wsiworld.com/global-expertise', 'https://www.wsiworld.com/loc

In [25]:
def get_relevant_links(full_links):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_links},
            {"role": "user", "content": f"here are the links: {full_links}"}
        ],
        response_format={"type": "json_object"})
    return json.loads(response.choices[0].message.content)  

relevant_links = get_relevant_links(links)   

In [26]:
#just so we can see the links
# print(relevant_links)

In [27]:
print(relevant_links.values())

# Extract the list of URLs
links_list = list(relevant_links.values())[0]  # Extract the first (and only) value

# Iterate over each link
for link in links_list:
    print(link)  # Now each URL is printed separately

dict_values([['https://www.wsiworld.com/blog/how-ai-is-revolutionizing-seo/', 'https://www.wsiworld.com/blog', 'https://www.wsiworld.com/blog/tag/seo', 'https://www.wsiworld.com/blog/how-googles-ai-overview-is-affecting-your-seo', 'https://www.wsiworld.com/blog/how-to-future-proof-your-business-with-ai-powered-seo', 'https://www.wsiworld.com/blog/essential-steps-to-crafting-a-winning-seo-strategy', 'https://www.wsiworld.com/learn', 'https://www.wsiworld.com/learn/artificial-intelligence', 'https://www.wsiworld.com/customer-stories/case-studies', 'https://www.wsiworld.com/customer-stories', 'https://www.wsiworld.com/our-services/competitive-analysis', 'https://www.wsiworld.com/our-services/buyer-persona-development', 'https://www.wsiworld.com/our-services/content-marketing', 'https://www.wsiworld.com/our-services/inbound-marketing', 'https://www.wsiworld.com/our-services/social-media-marketing', 'https://www.wsiworld.com/our-services/web-analytics-and-reporting', 'https://www.wsiworld.c

In [28]:
#scrape the content of each link in the links_list

# print(len(links_list))
display(Markdown(main_summary.content))
#update links list to only have the links that are not the main page
links_list = [link for link in links_list if link != url]

#max 5 links, from link 1 to link 5 because the first link is usually the main page
links_list = links_list[1:6]

print(len(links_list))

for i in range(len(links_list)):
    response = requests.get(links_list[i], headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)
    print(f"link {i}: {links_list[i]}")
    #use gpt to cut out the irrelevant content and summarize the content
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_main},
            {"role": "user", "content": f"Here is the page content: {text}"}
        ],
    )
    link_summary = response.choices[0].message.content
    display(Markdown(link_summary))


#display the main summary and the link summaries

# Executive Summary: How AI is Revolutionizing SEO

Artificial Intelligence (AI) is fundamentally transforming the landscape of Search Engine Optimization (SEO), enhancing how digital marketers approach various aspects of SEO strategies. The integration of AI into SEO practices focuses on improving search algorithms, automating essential tasks, and enriching user experiences—leading to more effective digital marketing results.

## Key Areas of Transformation

### 1. Enhanced Search Engine Crawling and Indexing
- **Advanced Algorithms**: AI utilizes advanced algorithms that move beyond traditional keyword density metrics. It focuses on content quality, keyword context, and contextual relevance based on user engagement metrics.
- **Personalization**: By refining search results through a comprehensive analysis of user interactions, AI can deliver more tailored outputs.

### 2. Content Creation and Optimization
- **AI Content Tools**: Digital marketers leverage AI tools that can generate high-quality, optimized content at scale, utilizing insights on user behavior.
- **Keyword Management**: AI assists in identifying optimal keywords, optimizing meta tags, and ensuring content readability, ultimately boosting search rankings and reducing bounce rates.

### 3. Voice Search Optimization
- **Natural Language Processing (NLP)**: AI advances in NLP are driving the trend towards voice search, necessitating that marketers optimize content for conversational queries.
- **Structured Data Automation**: AI facilitates the implementation of structured data to enhance the visibility of content in voice search results.

### 4. Predictive Analytics
- **Understanding User Intent**: By analyzing user behaviors, AI helps marketers align content strategies with user intent, attracting relevant traffic.
- **Forecasting Trends**: AI tools can predict future trends and user behaviors, enabling marketers to stay ahead in a dynamic SEO environment.

### 5. Improved User Experience (UX)
- **UX Optimization Suggestions**: AI analyzes user behavior to suggest enhancements that improve the overall user experience on websites.
- **Personalized Recommendations**: By tailoring content recommendations to user preferences, AI helps ensure users find relevant information quickly, potentially increasing engagement time.

## Conclusion
AI has become a game-changer in the realm of SEO, enabling marketers to create more effective, data-driven strategies. The ability to harness insights from AI not only optimizes search engine interactions but also enhances user engagement and experience, ultimately driving better marketing outcomes. As AI continues to evolve, its influence on SEO will likely expand, making it essential for marketers to embrace these technologies.

5
link 0: https://www.wsiworld.com/blog


# Executive Summary on Digital Marketing Trends

The content discusses various aspects of digital marketing, focusing on best practices, emerging technologies, and the importance of tailoring strategies to meet specific client needs.

## Key Components of Digital Marketing:

1. **Customization Over Standardization:**
   - The importance of avoiding cookie-cutter approaches to digital marketing is emphasized. Customized strategies can lead to better engagement and conversion.

2. **Services Offered:**
   - A range of services is available, including lead generation, website design, AI consulting, and brand awareness initiatives. Each service aims to leverage current technologies and techniques to enhance client performance.

3. **Emerging Trends:**
   - The blog highlights the pivotal role of AI in transforming marketing strategies. It discusses how AI-driven tools can revolutionize search marketing and optimize SEO practices, reflecting ongoing changes in consumer behavior and technology usage.

4. **Content Marketing:**
   - Strategies for effective content marketing are detailed, including the significance of landing pages and the impact of video content. The shift towards video-first content is noted, as it increases discoverability online.

5. **Customer Experience:**
   - The discussion also covers customer experience, emphasizing web accessibility as a crucial element. Creating user-friendly experiences can significantly enhance customer engagement and satisfaction.

6. **Educational Resources:**
   - A rich repository of resources is available to help marketers enhance their knowledge, encompassing ebooks, videos, and more, demonstrating a commitment to continuous learning in the evolving digital landscape.

7. **Community and Collaboration:**
   - The organization positions itself as a collaborative partner in the digital marketing field, encouraging consultations to offer tailored marketing strategies driven by global insights.

In summary, the content from this digital marketing platform underscores the necessity of personalized approaches, the integration of AI tools, and a strong emphasis on educational resources to navigate the rapidly changing digital marketing environment effectively.

link 1: https://www.wsiworld.com/blog/tag/seo


# Executive Summary: SEO and Its Evolution with AI

The content on the analyzed webpage discusses various aspects of Search Engine Optimization (SEO), particularly in the context of advancements in artificial intelligence (AI). It outlines the transformation that SEO is undergoing due to AI technologies, highlighting significant themes and practical strategies for businesses looking to adapt to these changes.

## Key Topics Covered

### 1. **AI and SEO Revolution**
   - The integration of AI into SEO is reshaping how algorithms function, improving the efficiency and effectiveness of search processes. AI enhances user experience by enabling more accurate search results and personalized content recommendations.

### 2. **Future-Proofing Businesses**
   - The articles stress the importance of adopting AI-powered SEO strategies to stay competitive. Businesses are encouraged to leverage AI tools to optimize their marketing efforts, thus achieving outcomes that traditional methods may not produce.

### 3. **Crafting an Effective SEO Strategy**
   - The webpage provides essential steps for developing a robust SEO strategy that encompasses goal setting, conducting audits, and utilizing AI for optimization. A structured approach to these elements can significantly enhance online visibility.

### 4. **Trending Topics in SEO**
   - Recent discussions in the SEO space include concepts like "Search Generative Experience" (SGE) and the impact of zero-click searches. These trends require businesses to remain agile and informed about how search behavior is evolving.

### 5. **Resources for Learning**
   - The site offers various resources, including articles and case studies, aimed at educating readers about contemporary SEO practices and the necessity for adaptation in digital marketing strategies.

### Conclusion
The overarching theme is the critical intersection of AI and SEO, providing businesses with both challenges and opportunities. The continual evolution of search algorithms and user interaction necessitates strategic adjustments to ensure optimal digital presence. By embracing these changes and utilizing available resources, organizations can enhance their lead generation and overall marketing effectiveness.

...

# How Google’s AI Overviews Are Shaping SEO

## Introduction to AI Overviews
Google's AI Overviews (AIO) is a new search feature that leverages artificial intelligence to provide direct answers to user queries by summarizing information from various online sources. Instead of relying solely on keywords, AIO aims to understand the intent behind queries and deliver relevant answers, significantly impacting search engine optimization (SEO) strategies.

## Key Implications for SEO

### Enhanced Content Understanding
The introduction of AIO has improved Google's ability to comprehend the context and intent behind digital content. This advancement means that content creators must develop high-quality, in-depth material that addresses user queries effectively, as generalized or thin content is less likely to rank well.

### Importance of E-E-A-T
AIO enhances Google's assessment of content based on the principles of Experience, Expertise, Authoritativeness, and Trustworthiness (E-E-A-T). To effectively leverage AIO, it is essential to produce content authored by qualified experts, incorporate reputable references, and maintain robust content quality signals. Additionally, maintaining a secure website (HTTPS) and providing clear privacy policies can further enhance trust with users.

### Focusing on Content Quality
AIO prioritizes comprehensive content over superficial entries. To adapt, businesses should focus on creating rich, informative content that satisfies both Google’s criteria and user expectations. Regular updates to existing content are also crucial to maintain relevance.

### Personalization in Search Results
AIO tailors search results based on user behavior, location, and preferences. Businesses should employ local SEO tactics such as optimizing Google Business Profiles, using localized keywords, and engaging with the local community via social media to enhance visibility.

### Utilizing Topic Clusters
AI's ability to understand semantic intent supports the creation of content structured around topic clusters. This involves using internal links and addressing related subtopics to boost overall context relevance and improve search rankings.

### Combatting Spam and Manipulative Tactics
AIO’s sophisticated AI can identify and penalize manipulative SEO practices like keyword stuffing. Ethical SEO practices, such as building backlinks through genuine content and relationships, are increasingly vital for maintaining rankings.

### Visual and Multimodal Search
With improved visual search capabilities, AIO processes various content forms simultaneously. To optimize this multimedia content, it's important to use proper metadata, image attributes, and ensure the quality of visual assets.

### Managing Zero-Click Content
Zero-click content answers queries within the search interface, enhancing user experience while minimizing direct website traffic. Effective strategies for zero-click content include optimizing for featured snippets, maintaining an active social media presence, and providing engaging content that encourages user interaction.

## Future Considerations
As Google continues to evolve AIO, staying updated on changes and adjustments is crucial. Businesses are encouraged to adapt their SEO strategies accordingly, ensuring they remain competitive in this shifting landscape.

## Conclusion
The integration of Google’s AI Overviews represents a significant shift in how SEO is approached. Quality, relevance, personalization, and ethical practices will be paramount to thriving in this new SEO environment shaped by AI capabilities. For those preparing for the future of SEO, embracing these changes and adapting strategies accordingly will be essential.

link 3: https://www.wsiworld.com/blog/how-to-future-proof-your-business-with-ai-powered-seo


# Executive Summary: Future-Proofing Your Business with AI-Powered SEO

The integration of artificial intelligence (AI) into search engine optimization (SEO) is transforming the digital marketing landscape. AI-powered SEO leverages advanced algorithms, machine learning, and language processing to enhance the effectiveness and efficiency of SEO strategies, ultimately helping businesses achieve higher search engine rankings.

## Key Benefits of AI-Powered SEO

1. **Enhanced Efficiency**: AI tools can analyze web pages to provide highly precise recommendations for improving search rankings much faster than traditional methods.
2. **Content Creation**: AI accelerates content generation, allowing marketers to produce personalized and engaging material tailored to specific audiences.
3. **Idea Generation**: AI can brainstorm new content ideas and find opportunities for link-building and technical SEO improvements.

## The Rise of AI Overviews and Zero-Click Searches

Google's AI Overviews and zero-click searches are significant developments in SEO. AI Overviews provide summarized answers to user queries directly on the search engine results page (SERP), thereby increasing content visibility but potentially reducing traffic to the actual website. Zero-click searches allow users to obtain answers without clicking through to the source, indicating a shift in user behavior and engagement patterns.

### Strategies to Adapt:

- **Quality Content Focus**: Instead of prioritizing keyword optimization, creating authoritative, engaging content that addresses user needs should be the main goal.
- **User Experience Improvement**: A positive user experience will help boost engagement metrics and ultimately improve search rankings.
- **Video Content Utilization**: Prioritizing video content can enhance discoverability across various platforms and cater to evolving search behaviors as search engines improve in indexing multimedia formats.

## Understanding Search Intent

A critical aspect of AI-powered SEO is grasping user search intent, categorized as informational, navigational, commercial, or transactional. Understanding these intents allows businesses to create targeted content that meets users' needs and enhances their overall experience.

## Creating “Pull-Through” Content

To combat challenges posed by AI Overviews and zero-click searches, businesses should focus on developing "pull-through" content that is too detailed or nuanced for AI summaries. This type of content should engage users deeply, encouraging them to visit the source site for further insights.

## Engaging Users with Interactive Tools

AI can also enhance user engagement through interactive content such as calculators, quizzes, and configurators, all of which are easily created with AI tools. These elements improve user interaction and can drive conversions effectively.

## Conclusion

Future-proofing SEO strategies with AI is essential for businesses to remain competitive. By leveraging AI for data analysis, understanding user intent, and continuously adapting content strategies, businesses can ensure they thrive in a rapidly evolving digital marketing environment. Maintaining a focus on quality, originality, and user experience will position businesses for greater success in the age of AI-driven SEO.

link 4: https://www.wsiworld.com/blog/essential-steps-to-crafting-a-winning-seo-strategy


# Executive Summary: Essential Steps to Crafting a Winning SEO Strategy

In today's digital landscape, a robust Search Engine Optimization (SEO) strategy is crucial for businesses of all sizes to establish an online presence, build credibility, and drive traffic. Here are the essential steps for creating an effective SEO strategy:

## 1. Define Your Goals
Begin with specific, measurable, achievable, relevant, and time-bound (SMART) goals that align with your overall business objectives. Common goals include increasing organic traffic, enhancing engagement, and improving conversion rates.

## 2. Perform a Comprehensive SEO Audit
Conduct an audit to assess your current SEO standing. Evaluate your website's content quality, backlink profile, site architecture, and ensure proper configuration of technical elements like robots.txt and sitemaps.

## 3. Conduct Keyword Research
Identify relevant, high-volume, low-competition keywords using research tools. Focus on understanding user intent to effectively target your audience. Long-tail keywords are particularly effective in attracting specific traffic at different stages of the conversion funnel.

## 4. Analyze the Competition
Perform a competitive analysis to identify your competitors' strengths and weaknesses. Utilize tools to explore their keyword rankings and backlink profiles, which can reveal optimization opportunities for your own strategy.

## 5. Develop a Content Strategy
Create a content strategy that builds authority and engages your target audience. Define your goals, understand your customer profiles, audit existing content, and set a realistic content calendar. Optimize content for SEO by naturally incorporating keywords and enhancing overall readability.

## 6. Build a Link-Building Strategy
Quality backlinks are essential for improving domain authority. Create shareable content, reach out to website owners for backlinks, and improve site usability to attract authoritative links.

## 7. Leverage AI for Optimization
AI technologies can significantly enhance SEO efforts. Automation of keyword research, competitor analysis, content optimization, and technical audits allows SEO professionals to focus on strategic tasks. AI can also aid in predicting trends and improving local SEO approaches.

## Conclusion
Crafting a winning SEO strategy requires careful planning, continuous analysis, and committed execution. By following these steps and incorporating AI, businesses can optimize their SEO efforts for sustained growth and improved online visibility. Remember that SEO is a long-term commitment; begin today to enhance your online presence and achieve your desired results.